### Dreambooth 模型微调
DreamBooth 是一种深度学习生成模型，用于微调现有的文本到图像模型，由 Google Research 和波士顿大学的研究人员于 2022 年开发。最初使用 Google 自己的 Imagen 文本到图像模型开发，DreamBooth 的实现可以应用到其他文本到图像模型，它可以让模型通过的三到五张图像对一个主题进行训练后生成更精细和个性化的输出。

![](../../images/dreambooth.png)

接下来我们将使在笔记本中docker中使用DreamBooth 来微调我们的 stable diffusion 模型.

#### 微调步骤
1. 设置train.sh,微调参数
2. 准备微调图片
3. 构建 dreambooth fine-tuning 镜像
4. 使用docker 运行微调任务
5. 查看微调结果
6. 部署微调模型到SageMaker Endpoint

In [ ]:
import time
import boto3
import sagemaker
account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

print(bucket)

#### 1. 参数设置
所有参数都在train.sh中

关键参数
   * --class_prompt "photo of a man"
   * --instance_prompt "photo of a zwx  man"
   * --max_train_steps 300
   * --learning_rate 2e-06

可以根据需要进行修改

In [ ]:
!cat train.sh

#### 2. 准备微调的图片
请下载预先准备的图片

In [ ]:
#请以现场指导人员提供的图片地址为准
!curl https://dfjcgkift2mhn.cloudfront.net/aigc/images.tgz | tar -xz 
!ls images/

#### 3. 构建 dreambooth fine-tuning 镜像
  使用预先定义的镜像，将修改过的train.sh和image 复制到镜像中

In [ ]:
!./build_push.sh

#### 4. 模型微调

   打开终端运行
  
   ```bash
      docker run --runtime=nvidia -e NVIDIA_VISIBLE_DEVICES=all  -ti sd-dreambooth-finetuning-v2 train.sh
   ```
   * 如果训练中间出现了空间不足，可以使用以下代码将/var/lib/docker 迁移到/home/ec2-user/SageMaker目录
   ```bash
       sudo systemctl stop docker
       sudo systemctl stop docker.socket 
       sudo mv /var/lib/docker /home/ec2-user/SageMaker 
       sudo ln -s /home/ec2-user/SageMaker/docker /var/lib/docker 
       sudo systemctl start docker.socket
       sudo systemctl start docker
   ```
  

#### 5. 查看微调效果测试
  

In [ ]:
import json
import io
from PIL import Image
import traceback
import time
from sagemaker.async_inference.waiter_config import WaiterConfig


s3_resource = boto3.resource('s3')

def get_bucket_and_key(s3uri):
    pos = s3uri.find('/', 5)
    bucket = s3uri[5 : pos]
    key = s3uri[pos + 1 : ]
    return bucket, key

def draw_image(response):
    try:
        bucket, key = get_bucket_and_key(response)
        print(bucket,key)
        obj = s3_resource.Object(bucket, key)
        bytes = obj.get()['Body'].read()
        image = Image.open(io.BytesIO(bytes))
        image.show()
    except Exception as e:
        traceback.print_exc()
        print(e)



In [ ]:
#请修改model_floder
model_floder="dreambooth/model/a21e22fe-b625-45ab-bde5-e080f1ff48c2"
for count in range(1,6):
    if model_floder!="":
        print(f's3://{bucket}/{model_floder}/samples/sample-{count}.jpg')
        draw_image(f's3://{bucket}/{model_floder}/samples/sample-{count}.jpg')

#### 6. 部署微调模型到SageMaker Endpoint
  

In [ ]:
#回到4.1 部署Stable Diffusion 模型 ,将andite/anything-v4.0替换为你fine tuning后的模型
framework_version = '1.10'
py_version = 'py38'

model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
    #'model_name':'andite/anything-v4.0',
    'model_name':'s3://bucket/<model floder>',
    's3_bucket':bucket
}

print(f's3://{bucket}/{model_floder}/samples/sample-{count}.jpg')